I study the apparent proper motion of radio sources to estimate the ICRF axis stability.

In [1]:
from astropy.table import Table
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np

# My progs
from my_progs.catalog.read_icrf import read_icrf3
# from my_progs.stat_func.rms_calc import rms_calc
from tool_func import calc_orient_4_yearly_crf1

In [2]:
icrf3sx = read_icrf3(wv="sx")
icrf3_sou = Table([icrf3sx["iers_name"]])

icrf3def = icrf3sx[icrf3sx["type"] == "D"]
icrf3_defsou = Table([icrf3def["iers_name"]])

In [3]:
data = Table.read("../logs/ts_stat.log", format="ascii.csv")

In [4]:
fig, ax = plt.subpl

AttributeError: module 'matplotlib.pyplot' has no attribute 'subpl'

In [ ]:
data1 = data.group_by("year")

ind0 = data1.groups.indices

ind = ind0[1:] - ind0[:-1]

In [ ]:
fig, ax = plt.subplots()

ax.plot(epoch, ind, "-o")
ax.set_xlabel("Epoch (year)", fontsize=12)
ax.set_ylabel("No. sources", fontsize=12)

ax.set_yscale("log")

# All sources

In [ ]:
# years = np.arange(1985, 2020)
# num = len(years)

# N0 = np.zeros_like(years)
# N1 = np.zeros_like(years)
# pmt = np.empty((num, 3), dtype=np.float)
# err = np.empty((num, 3), dtype=np.float)

In [ ]:
# for i in range(num):
    
#     print("\nProcessing", year)
    
#     pos_oft = data1.groups[i]
#     pos_oft = join(pos_oft, icrf3_sou)
    
#     N0[i], N1[i], pmt[i], err[i] = calc_orient_4_yearly_crf1(pos_oft)

In [ ]:
# fig, ax = plt.subplots()

# ax.plot(years, N0, "b-o", ms=2, lw=0.5, label="$N_0$")
# ax.plot(years, N1, "r-s", ms=2, lw=0.5, label="$N_1$")

# # ax.set_yscale("log")

# ax.set_xlabel("Epoch (year)", fontsize=12)
# ax.set_ylabel("No. sources", fontsize=12)
# ax.legend()

In [ ]:
# fig, ax = plt.subplots()

# ax.errorbar(years, pmt[:,0], yerr=err[:,0], color="blue", 
#             ms=3, fmt="-s", elinewidth=0.5, lw=0.2, label="$R_1$")
# ax.errorbar(years, pmt[:,1], yerr=err[:,1], color="black", 
#             ms=3, fmt="-o", elinewidth=0.5, lw=0.2, label="$R_2$")
# ax.errorbar(years, pmt[:,2], yerr=err[:,2], color="red", 
#             ms=3, fmt="-^", elinewidth=0.5, lw=0.2, label="$R_3$")

# ax.set_xlabel("Epoch (year)", fontsize=12)
# ax.set_ylabel("Orientation angle ($\mathrm{\mu as}$)", fontsize=12)
# ax.legend()

In [ ]:
# wmean1, wrms1, wstd1 = rms_calc(pmt[:,0])
# wmean2, wrms2, wstd2 = rms_calc(pmt[:,1])
# wmean3, wrms3, wstd3 = rms_calc(pmt[:,2])

# print("No weighted")
# print("    Mean   RMS  Std")
# print("    uas    uas  uas")
# print("R1  {:+4.0f}  {:4.0f}  {:.0f}".format(wmean1, wrms1, wstd1))
# print("R2  {:+4.0f}  {:4.0f}  {:.0f}".format(wmean2, wrms2, wstd2))
# print("R3  {:+4.0f}  {:4.0f}  {:.0f}".format(wmean3, wrms3, wstd3))

In [ ]:
wmean1, wrms1, wstd1 = rms_calc(pmt[:,0], err[:,0])
wmean2, wrms2, wstd2 = rms_calc(pmt[:,1], err[:,1])
wmean3, wrms3, wstd3 = rms_calc(pmt[:,2], err[:,2])

print("Weighted")
print("    Mean  WRMS Std")
print("    uas   uas  uas")
print("R1  {:+4.0f}  {:.0f}  {:.0f}".format(wmean1, wrms1, wstd1))
print("R2  {:+4.0f}  {:.0f}  {:.0f}".format(wmean2, wrms2, wstd2))
print("R3  {:+4.0f}  {:.0f}  {:.0f}".format(wmean3, wrms3, wstd3))

# ICRF3 defining sources only

In [ ]:
N0_d = np.zeros_like(years)
N1_d = np.zeros_like(years)
pmt_d = np.empty((num, 3), dtype=np.float)
err_d = np.empty((num, 3), dtype=np.float)

In [ ]:
for i, year in enumerate(years):
    
    print("\nProcessing", year)
    
    pos_oft = data1.groups[i]
    pos_oft = join(pos_oft, icrf3_defsou)
    
    N0[i], N1[i], pmt[i], err[i] = calc_orient_4_yearly_crf1(pos_oft)

In [ ]:
fig, ax = plt.subplots()

ax.plot(years, N0_d, "b-o", ms=2, lw=0.5, label="$N_0$")
ax.plot(years, N1_d, "r-s", ms=2, lw=0.5, label="$N_1$")

# ax.set_yscale("log")

ax.set_xlabel("Epoch (year)", fontsize=12)
ax.set_ylabel("No. sources", fontsize=12)
ax.legend()

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(years, pmt_d[:,0], yerr=err_d[:,0], color="blue", 
            ms=3, fmt="-s", elinewidth=0.5, lw=0.2, label="$R_1$")
ax.errorbar(years, pmt_d[:,1], yerr=err_d[:,1], color="black", 
            ms=3, fmt="-o", elinewidth=0.5, lw=0.2, label="$R_2$")
ax.errorbar(years, pmt_d[:,2], yerr=err_d[:,2], color="red", 
            ms=3, fmt="-^", elinewidth=0.5, lw=0.2, label="$R_3$")

ax.set_xlabel("Epoch (year)", fontsize=12)
ax.set_ylabel("Orientation angle ($\mathrm{\mu as}$)", fontsize=12)
ax.legend()

In [ ]:
wmean1, wrms1, wstd1 = rms_calc(pmt_d[:,0])
wmean2, wrms2, wstd2 = rms_calc(pmt_d[:,1])
wmean3, wrms3, wstd3 = rms_calc(pmt_d[:,2])

print("No weighted")
print("    Mean   RMS  Std")
print("    uas    uas  uas")
print("R1  {:+4.0f}  {:4.0f}  {:.0f}".format(wmean1, wrms1, wstd1))
print("R2  {:+4.0f}  {:4.0f}  {:.0f}".format(wmean2, wrms2, wstd2))
print("R3  {:+4.0f}  {:4.0f}  {:.0f}".format(wmean3, wrms3, wstd3))

In [ ]:
wmean1, wrms1, wstd1 = rms_calc(pmt_d[:,0], err_d[:,0])
wmean2, wrms2, wstd2 = rms_calc(pmt_d[:,1], err_d[:,1])
wmean3, wrms3, wstd3 = rms_calc(pmt_d[:,2], err_d[:,2])

print("Weighted")
print("    Mean  WRMS Std")
print("    uas   uas  uas")
print("R1  {:+4.0f}  {:.0f}  {:.0f}".format(wmean1, wrms1, wstd1))
print("R2  {:+4.0f}  {:.0f}  {:.0f}".format(wmean2, wrms2, wstd2))
print("R3  {:+4.0f}  {:.0f}  {:.0f}".format(wmean3, wrms3, wstd3))